<h1>Conceito</h1>

<h2>Aleatoriedade inicial</h2>

Os modelos de *machine learning* estão sujeitos à uma aleatoriedade inicial, principalmente pela maneira como os dados de treino e teste são divididos, mas também em alguns casos devido aos próprios modelos. Por exemplo:

- **Divisão treino/teste**: o módulo *train_test_split* do *SKLearn* divide os pontos de dados em treino e teste de maneira aleatória, 'embaralhando' os mesmos antes da divisão
- **Árvores de decisão**: em cada divisão as características são permutadas de maneira aleatória
- **Máquina de vetor de suporte**: pode 'embaralhar' aleatóriamente os pontos de dados de treinos, o que eventualmente resultará em vetores diferentes

Devido à isso, a técnica de *holdout* (separar a base de dados em dados de treino e dados de teste que serão usados para validação do modelo) é sujeita à interferência dessa aleatóriedade inicial, podendo apresentar diferentes resultados. Portanto, é importante definir outro método de validação dos modelos, que possa fornecer um intervalo para as métricas como acurácia, precisão etc.

<h2>Validação Cruzada</h2>

A validação cruzada consiste em dividir a base de dados de treino e teste em $n$ maneiras diferentes, e então realizando o procedimento de treino e teste para todas essas divisões. Então, através da média das métricas e do desvio padrão das mesas para essas amostras é possível definir um intervalo de confiança para cada uma delas.

<h2><i>Splitters</i> do <i>SKLearn</i></h2>

O método da biblioteca *SKLearn* que realiza a validação cruzada divide os dados sempre da mesma maneira. Por isso existem métodos na biblioteca que realizam essa divisão:

- ***KFold***: divide os dados em $n$ partições consecutivas, podendo 'embaralhar' os dados caso isso seja específicado.
- ***StratifiedKFold***: semelhante ao *KFold*, porém realiza a estratificação dos dados - ou seja, mantém as divisões com valores proporcionais de cada classe.
- ***GroupKFold***: semelhante ao *KFold*, porém garante que todos os elementos de um mesmo grupo especificado se mantenham no mesma divisão (treino ou teste)

<h1>Prática</h1>

In [16]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [3]:
# Lendo a base de dados a ser utilizada
dados = pd.read_csv(r'Dados\Base.csv')

In [4]:
# Dividindo inputs e outputs
x = dados[["preco", "idade_do_modelo", "km_por_ano"]].values
y = dados["vendido"].values.ravel()

In [5]:
# Dividindo treino e teste
SEED = 158020
np.random.seed(SEED)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)

In [6]:
# Verificando a acurácia de um modelo dummy, para definição de uma baseline
dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(x_train, y_train)
acuracia = dummy_stratified.score(x_test, y_test) * 100

print(f"A acurácia do dummy stratified foi de {acuracia: .2f}")

A acurácia do dummy stratified foi de  50.96


In [7]:
# Verificando a acurácia de um modelo de árvore de decisão:
SEED = 158020
np.random.seed(SEED)
modelo_dtc1 = DecisionTreeClassifier(max_depth=2)
modelo_dtc1.fit(x_train, y_train)
previsoes_dtc1 = modelo_dtc1.predict(x_test)

acuracia_dtc1 = accuracy_score(y_test, previsoes_dtc1) * 100
print (f"A acurácia foi {acuracia_dtc1: .2f}")

A acurácia foi  71.92


In [8]:
# Verificando a acurácia do mesmo modelo para outro valor de SEED:

SEED2 = 5
np.random.seed(SEED2)

x_train2, x_test2, y_train2, y_test2 = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)

modelo_dtc2 = DecisionTreeClassifier(max_depth=2)
modelo_dtc2.fit(x_train2, y_train2)
previsoes_dtc2 = modelo_dtc2.predict(x_test2)

acuracia_dtc2 = accuracy_score(y_test2, previsoes_dtc2) * 100
print (f"A acurácia foi {acuracia_dtc2: .2f}")

A acurácia foi  76.84


In [9]:
# Agora verificando o desempenho do modelo através da validação cruzada
SEED = 158020
np.random.seed(SEED)

modelo_dtc3 = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo_dtc3, x, y, cv=10)

resultados

{'fit_time': array([0.01049924, 0.00950122, 0.01100016, 0.01000237, 0.00899577,
        0.00999856, 0.01699901, 0.00899673, 0.0134995 , 0.00899839]),
 'score_time': array([0.00150013, 0.00138736, 0.00101304, 0.00150061, 0.00300574,
        0.00100255, 0.00151253, 0.00150251, 0.00100255, 0.00100064]),
 'test_score': array([0.742, 0.77 , 0.749, 0.764, 0.761, 0.764, 0.754, 0.755, 0.759,
        0.76 ])}

In [11]:
# Considerando que o test_score possuí uma distribuição normal e que, sendo assim, 95% das ocorrências estarão dentro do intervalo
# média - 2 desvios padrões e média + 2 desvios padrões, pode-se definir o intervalo da seguinte maneira:

media = resultados['test_score'].mean()*100
desvio_padrao = resultados['test_score'].std()*100

print(f'A acurácia com validação cruzada, para um nível de confiança de 95%, está entre {media - desvio_padrao:.2f}% e {media + desvio_padrao:.2f}%')

A acurácia com validação cruzada, para um nível de confiança de 95%, está entre 75.01% e 76.55%


In [12]:
# Repetindo o teste para valores diferentes de SEED:

# Agora verificando o desempenho do modelo através da validação cruzada
SEED = 30
np.random.seed(SEED)

modelo_dtc4 = DecisionTreeClassifier(max_depth=2)
resultados2 = cross_validate(modelo_dtc4, x, y, cv=10)

media2 = resultados2['test_score'].mean()*100
desvio_padrao2 = resultados2['test_score'].std()*100

print(f'A acurácia com validação cruzada, para um nível de confiança de 95%, está entre {media2 - desvio_padrao2:.2f}% e {media2 + desvio_padrao2:.2f}%')

A acurácia com validação cruzada, para um nível de confiança de 95%, está entre 75.01% e 76.55%


Ou seja, utilizando a validação cruzada é possível reduzir a influência da aleatoriedade inicial. Porém, nos exemplos acima, foi removida a aleatoriedade da validação cruzada: o método está dividindo os dados sempre da mesma maneira. Para realizar o teste considerando aleatoriedade também nesta etapa, é possível utilizar o método Kfold:

In [15]:
# Utilizando o método KFold:

cv = KFold(n_splits=10, shuffle=True)
modelo_dtc5 = DecisionTreeClassifier(max_depth=2)
resultados3 = cross_validate(modelo_dtc5, x, y, cv=cv)

media3 = resultados3['test_score'].mean()*100
desvio_padrao3 = resultados3['test_score'].std()*100

print(f'A acurácia com validação cruzada e aleatorização através do KFold, para um nível de confiança de 95%, está entre {media3 - desvio_padrao3:.2f}% e {media3 + desvio_padrao3:.2f}%')

A acurácia com validação cruzada e aleatorização através do KFold, para um nível de confiança de 95%, está entre 75.17% e 76.39%


Para garantir que os dados serão divididos de maneira estratificada (proporcional às classes), é possível utilizar o método KFold estratificado:

In [17]:
# Utilizando o método StratifiedKFold:

cv2 = StratifiedKFold(n_splits=10, shuffle=True)
modelo_dtc6 = DecisionTreeClassifier(max_depth=2)
resultados4 = cross_validate(modelo_dtc6, x, y, cv=cv2)

media4 = resultados4['test_score'].mean()*100
desvio_padrao4 = resultados4['test_score'].std()*100

print(f'A acurácia com validação cruzada e aleatorização através do StratifiedKFold, para um nível de confiança de 95%, está entre {media4 - desvio_padrao4:.2f}% e {media4 + desvio_padrao4:.2f}%')

A acurácia com validação cruzada e aleatorização através do StratifiedKFold, para um nível de confiança de 95%, está entre 74.66% e 76.90%
